In [7]:
import audio_metadata
import csv
import json
import librosa
import math
import numpy as np
import os
#import pandas as pd
import random
import scipy.signal as signal
from time import time

mediadir = "W:\Music\FLAC"
folder_startswith = "any"#["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "a", "b", "c", "d", "e", "f", "g", "h", "i"] # last crash after jimi hendrix
include_rootfiles = False
ofdir = "outfiles-1s"
clip_dur = 1 # in seconds
nfft = 256
precision = 0 # decimal places
clips_per_song = 5
start_end_width = 0.1 # ignore clips from the first and last (start_end_width) % of the song (to avoid fade ins/outs)

#df = None
#df_inuse = False
metacolnames = ['label', 'filename', 'clipnum']
pixelcolnames = []
allcolnames = []
datarows = []

genres = {}

lastdir = None

if not os.path.exists(ofdir):
    os.mkdir(ofdir)

for root, dirs, files in os.walk(mediadir):
    if (lastdir is not None) and (lastdir != root) and (len(datarows) > 0):
        fp = os.path.normpath(lastdir)
        print(f"Writing new datafile {os.path.join(ofdir, '--'.join(fp.split(os.sep)[2:] + ['data.csv']))}")
        # Write the previous directory's dataframe, just for safety
        with open(os.path.join(ofdir, '--'.join(fp.split(os.sep)[2:] + ['data.csv'])), 'w', newline='') as data_outfile:
            dw = csv.DictWriter(data_outfile, fieldnames=allcolnames)
            dw.writeheader()
            dw.writerows(datarows)
        #df.to_csv(os.path.join(ofdir, '--'.join(fp.split(os.sep)[2:] + ['data.csv'])))
        with open(os.path.join(ofdir, '--'.join(fp.split(os.sep)[2:] + ['genres.json'])), "w") as genres_outfile:
            genres_outfile.write(json.dumps(genres))
        
        # Clear dataframe and re-init genres
        #df = None
        #df_inuse = False
        datarows = []
        genres = {}
        lastdir = root
    
    rootsplit = os.path.normpath(root).split(os.sep)
    if len(rootsplit) > 3 and (type(folder_startswith) == list) and (rootsplit[3].lower().startswith(tuple(folder_startswith))):
        # Ignoring this folder
        print(f'ignoring folder {rootsplit[3]}')
        continue

    if len(rootsplit) == 3 and not include_rootfiles:
        print('ignoring files in root dir')
        continue

    for filename in files:
        t_start = time()
        if filename.lower().endswith(('.mp3', '.flac', '.wav')) and not filename.startswith('._'):
            print(os.path.join(root, filename))
            try:
                md = audio_metadata.load(os.path.join(root, filename))
                print(f"Loaded in {round(time() - t_start, 2)}s")
                genre = None
                if 'genre' in md['tags']:
                    gs = [e for s in [t.split(';') for t in md['tags']['genre']] for e in s]
                    genre = gs[0]
                    for g in gs:
                        if g in genres:
                            genres[g] += 1
                        else:
                            genres[g] = 1
                else:
                    continue
                
                t_start = time()
                samples, sample_rate = librosa.load(os.path.join(root, filename))
                print(f"Librosa loaded in {round(time() - t_start, 2)}s")
                real_clip_dur = clip_dur * 1.1
                samples_per_clip = sample_rate * clip_dur
                real_samples_per_clip = sample_rate * real_clip_dur

                t_start = time()
                f, t, sxx = signal.spectrogram(samples, nfft=nfft, return_onesided=True)
                print(f"Spectrogram in {round(time() - t_start, 2)}s")

                tbin_width = len(samples) / len(t)
                song_dur = len(samples) / sample_rate
                num_clips = math.floor(song_dur / clip_dur)
                clip_sample_starts = [c * samples_per_clip for c in range(0, num_clips)]
                clip_tbin_starts = [round(s / tbin_width) for s in clip_sample_starts]
                tbins_per_clip = math.floor(samples_per_clip / tbin_width)
                clip_tbin_ends = [s + tbins_per_clip for s in clip_tbin_starts]

                Sxx_dB = 10*np.log10(sxx)

                #if df is None:
                if len(datarows) == 0:
                    pixelcolnames = [f'pixel{n}' for n in range(0, len(f) * tbins_per_clip)]
                    allcolnames = metacolnames + pixelcolnames
                    #df = pd.DataFrame([], columns=allcolnames)
                    #df_inuse = True
                    if lastdir is None:
                        lastdir = root


                ### Talked with Tyler on 4/20 after class, instead of concatenating df's row by row try appending dicts to a list then use the list to make the dataframe
                ### To do that, find a way to convert the np array to a dict. I've already got a list of column names, so it shouldn't be too hard
                ### Also use time.time to check how long things are running, might be able to prove it is that pd.concat step that's increasing in duration each time

                t_start = time()
                clip_idxs = []
                clip_idx_start = math.ceil(len(clip_tbin_starts) * start_end_width)
                clip_idx_end = math.floor(len(clip_tbin_starts) * (1-start_end_width))
                for c in range(clips_per_song):
                    idx = random.randint(clip_idx_start, clip_idx_end)
                    while idx in clip_idxs:
                        idx = random.randint(clip_idx_start, clip_idx_end)
                    clip_idxs.append(idx)
                print(clip_idxs)
                for c in clip_idxs:
                    t_startclip = time()
                    #print(Sxx_dB[:, clip_tbin_starts[c]:clip_tbin_ends[c]].shape, t[clip_tbin_starts[c]], t[clip_tbin_ends[c]])
                    Sxx_dB_flat = np.ravel(Sxx_dB[:, clip_tbin_starts[c]:clip_tbin_ends[c]])
                    Sxx_dB_round = np.round(Sxx_dB_flat, precision)
                    row = dict(zip(pixelcolnames, Sxx_dB_round))
                    #print(Sxx_dB_flat)
                    #df = pd.concat([df, pd.DataFrame(Sxx_dB_flat.reshape(1, -1), columns=pixelcolnames)])
                    #df.iloc[-1, df.columns.get_loc('label')] = genre
                    #df.iloc[-1, df.columns.get_loc('filename')] = filename
                    #df.iloc[-1, df.columns.get_loc('clipnum')] = c
                    row['label'] = genre
                    row['filename'] = filename
                    row['clipnum'] = c
                    datarows.append(row)
                    #print(f"\tAppended clip in {round(time() - t_startclip, 2)}s")
                print(f"Finished appending clips in {round(time() - t_start, 2)}s")

            except audio_metadata.UnsupportedFormat as ufe:
                pass
                #print(f"Error with {filename}, skipping")
            except audio_metadata.FormatError as fe:
                pass
                #print(f"Format error with {filename}, skipping")

if (lastdir is not None) and (len(datarows) > 0):
    fp = os.path.normpath(lastdir)
    print(f"Writing new datafile {os.path.join(ofdir, '--'.join(fp.split(os.sep)[2:] + ['data.csv']))}")
    # Write the previous directory's dataframe, just for safety
    #df.to_csv(os.path.join(ofdir, '--'.join(fp.split(os.sep)[2:] + ['data.csv'])))
    with open(os.path.join(ofdir, '--'.join(fp.split(os.sep)[2:] + ['data.csv'])), 'w', newline='') as data_outfile:
        dw = csv.DictWriter(data_outfile, fieldnames=allcolnames)
        dw.writeheader()
        dw.writerows(datarows)
    with open(os.path.join(ofdir, '--'.join(fp.split(os.sep)[2:] + ['genres.json'])), "w") as genres_outfile:
        genres_outfile.write(json.dumps(genres))

ignoring files in root dir
W:\Music\MP3s\Aerosmith\Aerosmith - Aerosmith - Angel.mp3
Loaded in 0.03s
Librosa loaded in 1.42s
Spectrogram in 0.12s
[84, 266, 259, 31, 272]
Finished appending clips in 0.01s
W:\Music\MP3s\Aerosmith\Aerosmith - Aerosmith - Back in the Saddle.mp3
Loaded in 0.03s


    divide by zero encountered in log10


Librosa loaded in 1.31s
Spectrogram in 0.11s
[53, 242, 84, 204, 196]
Finished appending clips in 0.01s
W:\Music\MP3s\Aerosmith\Aerosmith - Aerosmith - Big Ten Inch Record.mp3
Loaded in 0.03s
Librosa loaded in 0.66s
Spectrogram in 0.05s
[55, 97, 51, 100, 103]
Finished appending clips in 0.01s
W:\Music\MP3s\Aerosmith\Aerosmith - Aerosmith - Draw the Line.mp3
Loaded in 0.03s
Librosa loaded in 1.06s
Spectrogram in 0.08s
[183, 184, 85, 45, 42]
Finished appending clips in 0.01s
W:\Music\MP3s\Aerosmith\Aerosmith - Aerosmith - Dream On.mp3
Loaded in 0.03s
Librosa loaded in 1.24s
Spectrogram in 0.1s
[123, 119, 228, 57, 213]
Finished appending clips in 0.01s
W:\Music\MP3s\Aerosmith\Aerosmith - Aerosmith - Dude (Looks Like a Lady).mp3
Loaded in 0.03s
Librosa loaded in 1.23s
Spectrogram in 0.1s
[121, 197, 203, 180, 101]
Finished appending clips in 0.01s
W:\Music\MP3s\Aerosmith\Aerosmith - Aerosmith - Janie's Got a Gun.mp3
Loaded in 0.03s
Librosa loaded in 1.5s
Spectrogram in 0.12s
[163, 193, 181, 

    Ignoring ``TRCK``.
    Number frame values must consist only of digits and '/'.
    
    Ignoring ``TSOA`` frame with value ``['George Orcullo Ortha II +639209517019 Sycip Gorres Velayo & Co.']``.
    ``TSOA`` is not supported in the ID3v2.3 specification.
    
    Ignoring ``TRCK``.
    Number frame values must consist only of digits and '/'.
    
    Ignoring ``TSOA`` frame with value ``['George Orcullo Ortha II +639209517019 Sycip Gorres Velayo & Co.']``.
    ``TSOA`` is not supported in the ID3v2.3 specification.
    
    Ignoring ``TRCK``.
    Number frame values must consist only of digits and '/'.
    
    Ignoring ``TSOA`` frame with value ``['George Orcullo Ortha II +639209517019 Sycip Gorres Velayo & Co.']``.
    ``TSOA`` is not supported in the ID3v2.3 specification.
    
    Ignoring ``TRCK``.
    Number frame values must consist only of digits and '/'.
    
    Ignoring ``TSOA`` frame with value ``['George Orcullo Ortha II +639209517019 Sycip Gorres Velayo & Co.']``.
 

Loaded in 0.11s
W:\Music\MP3s\Blanchard's Friday Playlist\Ariana Grande - Everyday feat Future (Clean).mp3
Loaded in 0.05s
W:\Music\MP3s\Blanchard's Friday Playlist\Ariana Grande - Focus.mp3
Loaded in 0.04s
W:\Music\MP3s\Blanchard's Friday Playlist\Bebe Rexha - I Got You.mp3
Loaded in 0.03s
W:\Music\MP3s\Blanchard's Friday Playlist\Bruno Mars - 24k.mp3
Loaded in 0.03s
W:\Music\MP3s\Blanchard's Friday Playlist\Bruno Mars - Thats What I Like (Clean).mp3
Loaded in 0.03s
W:\Music\MP3s\Blanchard's Friday Playlist\Calvin Harris & Disciples - How Deep Is Your Love.mp3


...

Loaded in 0.11s
W:\Music\MP3s\Blanchard's Friday Playlist\twenty one pilots - Ride.mp3
Loaded in 0.11s
Librosa loaded in 0.5s
Spectrogram in 0.08s
[117, 62, 116, 120, 32]
Finished appending clips in 0.01s
W:\Music\MP3s\Blanchard's Friday Playlist\twenty one pilots - Stressed Out.mp3
Loaded in 0.1s
Librosa loaded in 0.89s
Spectrogram in 0.08s
[90, 52, 173, 91, 165]
Finished appending clips in 0.01s
W:\Music\MP3s\Blanchard's Friday Playlist\Zara Larsson & MNEK - Never Forget You.mp3


    Ignoring ``TRCK``.
    Number frame values must consist only of digits and '/'.
    
    Ignoring ``TSOA`` frame with value ``['George Orcullo Ortha II +639209517019 Sycip Gorres Velayo & Co.']``.
    ``TSOA`` is not supported in the ID3v2.3 specification.
    
    Ignoring ``TRCK``.
    Number frame values must consist only of digits and '/'.
    
    Ignoring ``TSOA`` frame with value ``['George Orcullo Ortha II +639209517019 Sycip Gorres Velayo & Co.']``.
    ``TSOA`` is not supported in the ID3v2.3 specification.
    


Loaded in 0.1s
W:\Music\MP3s\Blanchard's Friday Playlist\ZAYN Taylor Swift - I Don't Wanna Live Forever.mp3
Loaded in 0.04s
W:\Music\MP3s\Blanchard's Friday Playlist\Zedd ft Selena Gomez - I Want You To Know.mp3
Loaded in 0.11s


    Ignoring ``SYLT``.
    Missing data in synchronized lyrics frame.
    
    Ignoring ``SYLT``.
    Missing data in synchronized lyrics frame.
    


Librosa loaded in 0.9s
Spectrogram in 0.09s
[210, 126, 48, 123, 151]
Finished appending clips in 0.01s
Writing new datafile outfiles-1s\MP3s--Blanchard's Friday Playlist--data.csv
W:\Music\MP3s\junk\Booker T. & The M.G.'s - Melting Pot.mp3
Loaded in 0.14s
Librosa loaded in 1.13s
Spectrogram in 0.19s
[338, 198, 182, 276, 401]
Finished appending clips in 0.01s
W:\Music\MP3s\junk\Boston Horns - Funkafized.mp3
Loaded in 0.1s
Librosa loaded in 0.67s
Spectrogram in 0.12s
[239, 230, 69, 248, 191]
Finished appending clips in 0.01s
W:\Music\MP3s\junk\Buddy Rich - Nuttville.mp3
Loaded in 0.13s
Librosa loaded in 0.65s
Spectrogram in 0.11s
[236, 110, 208, 42, 220]
Finished appending clips in 0.01s
W:\Music\MP3s\junk\Casiopea - Asayake.mp3
Loaded in 0.11s
Librosa loaded in 0.68s
Spectrogram in 0.12s
[174, 218, 147, 99, 145]
Finished appending clips in 0.01s
W:\Music\MP3s\junk\Casiopea - Domino Line.mp3
Loaded in 0.12s
Librosa loaded in 0.67s
Spectrogram in 0.11s
[235, 196, 148, 176, 57]
Finished ap

In [5]:
# Collect all the data files into one large csv and genre json

import csv
import json
import os
from time import time

outdir = 'outfiles/'

genre_list = []
cols = None
dw = None

with open('data.csv', 'w', newline='') as of_data:
    for fp in os.listdir(outdir):
        t_start = time()
        print(fp)
        if fp.endswith('.csv'):
            with open(os.path.join(outdir, fp), 'r') as f:
                dr = csv.reader(f)
                if cols is None:
                    cols = next(dr)
                    dw = csv.writer(of_data)
                    dw.writerow(cols)
                else:
                    next(dr)
                row = next(dr, False)
                while (row != False):
                    dw.writerow(row)
                    row = next(dr, False)
            print(f"CSV in {round(time() - t_start, 2)}s")
        if fp.endswith('.json'):
            with open(os.path.join(outdir, fp), 'r') as f:
                genre_list.append(json.load(f))
            print(f"JSON in {round(time() - t_start, 2)}s")

# Combine genre dicts into one
genres = {}
for gd in genre_list:
    for g in gd:
        if g in genres:
            genres[g]['count'] = genres[g]['count'] + gd[g]
        else:
            genres[g] = {}
            genres[g]['count'] = gd[g]
            genres[g]['clean'] = ""

print(genres)
with open('genres.json', 'w') as of_genre:
    json.dump(genres, of_genre)


FLAC---NSYNC--data.csv
CSV in 8.66s
FLAC---NSYNC--genres.json
JSON in 0.01s
FLAC---NSYNC--Greatest Hits--data.csv
CSV in 9.6s
FLAC---NSYNC--Greatest Hits--genres.json
JSON in 0.0s
FLAC--10,000 Maniacs--data.csv
CSV in 11.63s
FLAC--10,000 Maniacs--genres.json
JSON in 0.0s
FLAC--123--data.csv
CSV in 53.73s
FLAC--123--genres.json
JSON in 0.0s
FLAC--3 Doors Down--data.csv
CSV in 8.63s
FLAC--3 Doors Down--genres.json
JSON in 0.0s
FLAC--30 Seconds To Mars--data.csv
CSV in 11.0s
FLAC--30 Seconds To Mars--genres.json
JSON in 0.0s
FLAC--3OH!3--data.csv
CSV in 0.41s
FLAC--3OH!3--genres.json
JSON in 0.0s
FLAC--3OH!3--Streets Of Gold--data.csv
CSV in 8.2s
FLAC--3OH!3--Streets Of Gold--genres.json
JSON in 0.0s
FLAC--3OH!3--Want [Deluxe]--data.csv
CSV in 8.92s
FLAC--3OH!3--Want [Deluxe]--genres.json
JSON in 0.0s
FLAC--60's Afternoon--data.csv
CSV in 10.89s
FLAC--60's Afternoon--genres.json
JSON in 0.0s
FLAC--60's Beach House--data.csv
CSV in 15.91s
FLAC--60's Beach House--genres.json
JSON in 0.0s
FL